In [42]:
import torch
from collections import Counter
from IoU import IntersectionOverUnion

def meanAveragePrecision(pred_boxes, true_boxes, iou_threshold=0.5, box_format="corners", num_classes=20):
    """
    AP@n = sum_i=1^N (P@i * (R@(i+1) - R@i)) = int P@i * S
    Input:
    pred_boxes: (list) predicted bounding boxes of all image all classes. formate as  
                [[train_idx, class_pred, prob_score, xyHW], [], []]
    true_boxes: (list) ture boxes of all image all classes. format as above
    iou_threshold: (float) if IoU(pred, targ)> iou_threshold, it is the True Positive(TP).
    box_format: (str) corners or centers same as IntersectionOverUnion function
    num_classes: (int) all classes in all images.
    Output:
    mean Average Precision: (float)
    """
    # pred_boxes (list): [[train_idx, class_pred, prob_score, xyHW], [], []]
    average_precisions = []
    epsilon = 1e-6

    for c in range(num_classes):
        detect_boxes = []
        exist_boxes = []
        
        # assign diff box to diff class.
        for det_box in pred_boxes:
            if det_box[1] == c:
                detect_boxes.append(det_box)

        # assign diff tgt box to diff class.
        for tgt_box in true_boxes:
            if tgt_box[1] == c:
                exist_boxes.append(tgt_box)


        # Counter return a dict, {key: num_occur}
        # img0 has 3 bbox
        # img1 has 5 bbox
        # amount_bboxes={0:3, 1:5}
        unmatched_trueboxes = Counter([tb[0] for tb in exist_boxes])
        
        for key, val in unmatched_trueboxes.items():
            unmatched_trueboxes[key] = torch.zeros(val)

        detect_boxes.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros(len(detect_boxes))
        FP = torch.zeros(len(detect_boxes))
        num_true_bboxes = len(exist_boxes)

        # If none exists for this class then we can safely skip
        if num_true_bboxes == 0:
            continue        

        # loop pred_box
        for det_idx, detect in enumerate(detect_boxes):
            iou_max = 0
            # in same image
            #if detect[0] == exist_boxes[0]:
            ground_true_boxes = [box for box in exist_boxes if detect[0] == box[0]]
            for truebox_idx, truebox in enumerate(ground_true_boxes):
                if unmatched_trueboxes[detect[0]][truebox_idx] == 0:
                    iou = IntersectionOverUnion(torch.tensor(detect[3:]),
                            torch.tensor(truebox[3:]), box_format=box_format)

                    if iou > iou_max:
                        iou_max = iou
                        truebox_max_idx = truebox_idx

            if iou_max > iou_threshold:
                TP[det_idx] = 1
                # detect[0] is the image idx
                unmatched_trueboxes[detect[0]][truebox_max_idx]=1
            else:
                FP[det_idx] = 1
        # cumsum TP, FP
        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        precision = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
        recall = TP_cumsum / (num_true_bboxes + epsilon)
        precision = torch.cat((torch.tensor([1]), precision))
        recall = torch.cat((torch.tensor([0]), recall))

        average_precisions.append(torch.trapz(precision, recall))

#         print(TP, FP, TP_cumsum, FP_cumsum, precision, recall, average_precisions)
        print(average_precisions)
            
    return sum(average_precisions) / len(average_precisions)


In [38]:
import torch
from collections import Counter
from IoU import IntersectionOverUnion

# from iou import intersection_over_union

def mean_average_precision(
    pred_boxes, true_boxes, iou_threshold=0.5, box_format="corners", num_classes=20
):
    """
    Calculates mean average precision 
    Parameters:
        pred_boxes (list): list of lists containing all bboxes with each bboxes
        specified as [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
        true_boxes (list): Similar as pred_boxes except all the correct ones 
        iou_threshold (float): threshold where predicted bboxes is correct
        box_format (str): "midpoint" or "corners" used to specify bboxes
        num_classes (int): number of classes
    Returns:
        float: mAP value across all classes given a specific IoU threshold 
    """

    # list storing all AP for respective classes
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6

    for c in range(num_classes):
        detections = []
        ground_truths = []

        # Go through all predictions and targets,
        # and only add the ones that belong to the
        # current class c
        for detection in pred_boxes:
            if detection[1] == c:
                detections.append(detection)

        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)

        # find the amount of bboxes for each training example
        # Counter here finds how many ground truth bboxes we get
        # for each training example, so let's say img 0 has 3,
        # img 1 has 5 then we will obtain a dictionary with:
        # amount_bboxes = {0:3, 1:5}
        amount_bboxes = Counter([gt[0] for gt in ground_truths])

        # We then go through each key, val in this dictionary
        # and convert to the following (w.r.t same example):
        # ammount_bboxes = {0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        # sort by box probabilities which is index 2
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)
      
        # If none exists for this class then we can safely skip
        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            # Only take out the ground_truths that have the same
            # training idx as detection
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            num_gts = len(ground_truth_img)
            best_iou = 0

            for idx, gt in enumerate(ground_truth_img):
                iou = IntersectionOverUnion(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[3:]),
                    box_format=box_format,
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        average_precisions.append(torch.trapz(precisions, recalls))
#         print('aladin', TP, FP, TP_cumsum, FP_cumsum, precisions, recalls, average_precisions)
        print(average_precisions)

    return sum(average_precisions) / len(average_precisions)

In [41]:
pred = [[1, 0, 0.9, 0.5, 0.5, 1.0, 1.2],
       [2, 0, 0.8, 0.5, 0.9, 1.1, 1.3],
       [1, 0, 0.7, 0.3, 0.2, 0.9, 1.6],
       [1, 0, 0.6, 0.6, 0.1, 1.9, 0.6],
       [3, 0, 0.3, 0.3, 0.9, 1.1, 0.1]]
ture = [[1, 0, 1., 0.5, 0.5, 0.9, 1.2],
        [2, 0, 1., 0.5, 0.9, 1.0, 1.2],
        [3, 0, 1., 0.3, 0.9, 1.0, 0.2]]
s1 = meanAveragePrecision(pred, ture)
s2 = mean_average_precision(pred, ture)
print(s1, s2)

[tensor(0.6667)]
[tensor(0.6667)]
tensor(0.6667) tensor(0.6667)
